In [55]:
import os
import pandas as pd
os.chdir('/home/data/yangz/bioinfo/packages/exprmat/exprmat/data/mmu/assemblies/grcm39')
os.getcwd()

'/home/data/yangz/bioinfo/packages/exprmat/exprmat/data/mmu/assemblies/grcm39'

In [56]:
ann = pd.read_table('annotations.gff.gz', comment = '#', sep = '\t', header = None)

In [57]:
ann

,0,1,2,3,4,5,6,7,8
0,chr1,HAVANA,gene,3143476,3144545,.,+,.,ID=ENSMUSG00000102693.2;gene_id=ENSMUSG0000010...
1,chr1,HAVANA,transcript,3143476,3144545,.,+,.,ID=ENSMUST00000193812.2;Parent=ENSMUSG00000102...
2,chr1,HAVANA,exon,3143476,3144545,.,+,.,ID=exon:ENSMUST00000193812.2:1;Parent=ENSMUST0...
3,chr1,ENSEMBL,gene,3172239,3172348,.,+,.,ID=ENSMUSG00000064842.3;gene_id=ENSMUSG0000006...
4,chr1,ENSEMBL,transcript,3172239,3172348,.,+,.,ID=ENSMUST00000082908.3;Parent=ENSMUSG00000064...
...,...,...,...,...,...,...,...,...,...
1312349,chrM,ENSEMBL,transcript,15289,15355,.,+,.,ID=ENSMUST00000082422.1;Parent=ENSMUSG00000064...
1312350,chrM,ENSEMBL,exon,15289,15355,.,+,.,ID=exon:ENSMUST00000082422.1:1;Parent=ENSMUST0...
1312351,chrM,ENSEMBL,gene,15356,15422,.,-,.,ID=ENSMUSG00000064372.1;gene_id=ENSMUSG0000006...
1312352,chrM,ENSEMBL,transcript,15356,15422,.,-,.,ID=ENSMUST00000082423.1;Parent=ENSMUSG00000064...


In [58]:
ann[2].value_counts()

2
exon                                      536952
CDS                                       428695
transcript                                 84484
five_prime_UTR                             69039
gene                                       56691
three_prime_UTR                            45584
start_codon                                45536
stop_codon                                 45315
stop_codon_redefined_as_selenocysteine        58
Name: count, dtype: int64

In [59]:
id = []
gene_id = []
subtype = []
parent = []
mapper = {
    'exon': 'exon',
    'UTR5': 'utr5',
    'UTR3': 'utr3',
    'CDS': 'cds',
    'UTR': 'utr',
    'stop_codon': 'stop',
    'start_codon': 'start',
    'selenocysteine': 'stop.sel'
}

from tqdm import tqdm 
for line, recordtype in tqdm(zip(ann[8].tolist(), ann[2].tolist())):
    terms = line.split(';')
    _id = ''
    _gene_id = ''
    _subtype = ''
    _parent = ''
    for term in terms:
        pair = term.split('=')
        
        if pair[0] == 'ID':
            if (
                pair[1].startswith('exon:') or
                pair[1].startswith('UTR5:') or
                pair[1].startswith('UTR3:') or
                pair[1].startswith('CDS:') or
                pair[1].startswith('UTR:') or
                pair[1].startswith('stop_codon:') or
                pair[1].startswith('start_codon:') or
                pair[1].startswith('selenocysteine:')
            ): 
                stems = pair[1].split(':')
                if len(stems) == 2:
                    _id = mapper[stems[0]] + ':1'
                else: _id = mapper[stems[0]] + ':' + stems[2]
            
            else: _id = pair[1]
        
        elif pair[0] == 'gene_id':
            _gene_id = pair[1]

        elif pair[0] == 'gene_type' and recordtype == 'gene':    
            _subtype = pair[1]
        
        elif pair[0] == 'transcript_type' and recordtype == 'transcript':
            _subtype = pair[1]
        
        elif pair[0] == 'Parent':
            _parent = pair[1]
    
    id.append(_id)
    gene_id.append(_gene_id)
    subtype.append(_subtype)
    parent.append(_parent)

hier = pd.DataFrame({
    'id': id,
    'gid': gene_id,
    'subtype': subtype,
    'parent': parent
})

1312354it [00:08, 152979.48it/s]


In [60]:
hier['id'] = [x.split('.')[0] for x in hier['id']]
hier['gid'] = [x.split('.')[0] for x in hier['gid']]
hier['parent'] = [x.split('.')[0] for x in hier['parent']]

In [61]:
hier['id'].is_unique

False

In [62]:
merged = pd.concat([
    ann[[0, 2, 3, 4, 6]],
    hier
], axis = 1)

In [63]:
merged['chr'] = ['MT' if x == 'chrM' else x.replace('chr', '') for x in merged[0]]
merged.columns = [
    'ucsc', 'type', 'start', 'end', 'strand',
    'id', 'gid', 'subtype', 'parent', 'chr'
]

In [65]:
merged.to_feather('gene-models.feather')

In [64]:
merged

,ucsc,type,start,end,strand,id,gid,subtype,parent,chr
0,chr1,gene,3143476,3144545,+,ENSMUSG00000102693,ENSMUSG00000102693,TEC,,1
1,chr1,transcript,3143476,3144545,+,ENSMUST00000193812,ENSMUSG00000102693,TEC,ENSMUSG00000102693,1
2,chr1,exon,3143476,3144545,+,exon:1,ENSMUSG00000102693,,ENSMUST00000193812,1
3,chr1,gene,3172239,3172348,+,ENSMUSG00000064842,ENSMUSG00000064842,snRNA,,1
4,chr1,transcript,3172239,3172348,+,ENSMUST00000082908,ENSMUSG00000064842,snRNA,ENSMUSG00000064842,1
...,...,...,...,...,...,...,...,...,...,...
1312349,chrM,transcript,15289,15355,+,ENSMUST00000082422,ENSMUSG00000064371,Mt_tRNA,ENSMUSG00000064371,MT
1312350,chrM,exon,15289,15355,+,exon:1,ENSMUSG00000064371,,ENSMUST00000082422,MT
1312351,chrM,gene,15356,15422,-,ENSMUSG00000064372,ENSMUSG00000064372,Mt_tRNA,,MT
1312352,chrM,transcript,15356,15422,-,ENSMUST00000082423,ENSMUSG00000064372,Mt_tRNA,ENSMUSG00000064372,MT
